In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import os
import time as time
import copy as copy

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from autoencoder import Autoencoder, Model


from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from scipy.optimize import linear_sum_assignment
from scipy.stats import spearmanr

In [3]:
XL_PATH = r"radiomicsFeatures.csv"
MASK_FEATS = ["id", "label"]

feats_df = pd.read_csv(XL_PATH)
feats_df.head()

,id,label,sub_wout_original_glcm_ClusterProminence,adc_original_firstorder_Minimum,sub_wout_original_glszm_LowGrayLevelZoneEmphasis,sub_wout_original_firstorder_Maximum,adc_original_glcm_ClusterShade,sub_wout_original_firstorder_Mean,sub_win_original_glcm_Autocorrelation,adc_original_glszm_LargeAreaLowGrayLevelEmphasis,...,sub_win_original_glszm_ZoneEntropy,t2w_original_glszm_SizeZoneNonUniformityNormalized,t2w_original_glcm_JointEntropy,t2w_original_glszm_LargeAreaHighGrayLevelEmphasis,sub_win_original_glszm_SizeZoneNonUniformityNormalized,sub_wout_original_glszm_SmallAreaHighGrayLevelEmphasis,sub_win_original_glcm_MaximumProbability,sub_win_original_glcm_Imc1,sub_wout_original_glcm_JointEntropy,t2w_original_glszm_LargeAreaLowGrayLevelEmphasis
0,2535039,1,4.677862e+06,0.0,0.003103,600.0,14835.837461,299.900214,3755.933491,0.010393,...,6.339939,0.286470,10.166389,27423.571919,0.461100,2946.837800,0.034622,-0.041978,10.452108,0.033786
1,2417361,0,4.834267e+06,0.0,0.001672,600.0,-17634.034850,299.918235,3941.494865,0.058145,...,7.424770,0.350004,11.649157,21732.551407,0.604518,3322.225544,0.002107,-0.109242,11.891117,0.009861
2,2602563,1,5.159220e+06,0.0,0.001600,600.0,-19736.430500,299.820687,2455.254084,0.019202,...,7.239270,0.350692,10.919838,15567.069802,0.574356,3407.597573,0.004002,-0.194449,11.214368,0.018991
3,2902440,0,3.613791e+06,0.0,0.002428,600.0,-12881.976888,299.240444,3954.079034,0.576021,...,7.454390,0.380537,11.530000,18389.243521,0.566131,3121.573712,0.004134,-0.116415,11.669841,0.007846
4,2921898,0,5.773968e+06,0.0,0.001720,600.0,2116.811733,299.983523,3793.819336,0.011764,...,6.755170,0.265413,9.504938,245786.779116,0.469149,3175.569089,0.027634,-0.058680,11.459667,0.024444


### Stratified CV Fold Construction

In [7]:
pids = feats_df.id.to_numpy()
labels = feats_df.label.to_numpy()

In [9]:
pids

array([2535039, 2417361, 2602563, 2902440, 2921898, 3039346, 3110297,
       3110706, 3137563, 3207798, 3213683, 3222346, 3226033, 3303911,
       3310301, 3325442, 3327697, 3329611, 3332798, 3336537, 3405013,
       3416781, 3419338, 3502691, 3504033, 3513664, 3519247, 3522629,
       3534419, 3534604, 3536230, 3605303, 3607842, 3610014, 3613524,
       3616819, 3618480, 3621681, 3621824, 3621917, 3622974, 3631910,
       3632788, 3701079, 3702147, 3702859, 3703425, 3707565, 3712766,
       3713983, 3714280, 3715560, 3716356, 3718385, 3720950, 3724846,
       3725583, 3726460, 3727030, 3727850, 3728041, 3729691, 3730269,
       3800022, 3802504, 3805884, 3808093, 3811134, 3811851, 3811967,
       3812057, 3815317, 3817381, 3819464, 3819781, 3821188, 3821859,
       3822353, 3823293, 3823428, 3825318, 3827579, 3828403, 3833691,
       3833806, 3900009, 3902242, 3901619, 3904119, 3904751, 3906071,
       3906505, 3907211, 3907314, 3907344, 3908895, 3911843, 9534972,
       9803775, 9816

### Dataset Preparation

In [ ]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, X):
        self.X = X
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return self.X[i], self.X[i]

### DSAE Architecture